# Using Microsoft SEAL on Android

With Xamarin, and the Microsoft SEAL NuGet package, we are able to to develop Android applications incorporating homomorphic encryption using .NET. But before we jump into the Android app development, we will begin with writing a simple C# program that will be transferred to the Android app. This program will consist of repeatedly encrypting, then decrypting a number, and measuring the time taken on average to encrypt and decrypt the number once. This is a very important time to know, as it can be the bottleneck in the frequency a certain operation can run with on a phone.

To begin with, we need to import some libraries that we will be using. We need to get the NuGet packaged for Microsoft SEAL, as well as the System.Diagnostics class which will help us in timing.

In [15]:
#r "nuget:Microsoft.Research.SEALNet, 3.5.1"
using Microsoft.Research.SEAL;

using System;
using System.Diagnostics;

Our simple program will consist of setting up the encryption parameters, then encrypting and decrypting a number inside a for loop. To do this, we simply set up the encryption parameters as is done in the example for the CKKS scheme in the [Microsoft SEAL examples](https://github.com/microsoft/SEAL/blob/master/dotnet/examples/4_CKKS_Basics.cs). We then encode a number, the mathematical constant *e* to the 12th decimal place, in this case, then encrypt it. We immediately decrypt it then decode it back to a floating point number.

In [46]:
class test{
    static void Main(){
        using EncryptionParameters parms = new EncryptionParameters(SchemeType.CKKS);
        ulong polyModulusDegree = 8192;
        parms.PolyModulusDegree = polyModulusDegree;
        parms.CoeffModulus = CoeffModulus.Create(
                        polyModulusDegree, new int[]{ 60, 40, 40, 60 });
        double scale = Math.Pow(2.0, 40);

        using SEALContext context = new SEALContext(parms);
        Console.WriteLine();

        using KeyGenerator keygen = new KeyGenerator(context);
        using PublicKey publicKey = keygen.PublicKey;
        using SecretKey secretKey = keygen.SecretKey;
        using Encryptor encryptor = new Encryptor(context, publicKey);
        using Decryptor decryptor = new Decryptor(context, secretKey);

        using CKKSEncoder encoder = new CKKSEncoder(context);

        using Plaintext xPlain = new Plaintext();
        using Plaintext plainResult = new Plaintext();
        using Ciphertext x1Encrypted = new Ciphertext();

        List<double> result = new List<double>();

        encoder.Encode(2.718281828459, scale, xPlain);
        encryptor.Encrypt(xPlain, x1Encrypted);
        decryptor.Decrypt(x1Encrypted, plainResult);
        encoder.Decode(plainResult, result);

        Console.WriteLine(2.718281828459);
        Console.WriteLine(result[0]);
    }
}

After running this to verify that the result is the same (or at least similar) to our original value, we can add code to display the time it took to encrypt and decrypt the number, using the [Stopwatch class](https://docs.microsoft.com/en-us/dotnet/api/system.diagnostics.stopwatch?view=netcore-3.1), and print out the number of milliseconds it takes to encrypt and decrypt once.

In [47]:
class test{
    static void Main(){
        using EncryptionParameters parms = new EncryptionParameters(SchemeType.CKKS);
        ulong polyModulusDegree = 8192;
        parms.PolyModulusDegree = polyModulusDegree;
        parms.CoeffModulus = CoeffModulus.Create(
                        polyModulusDegree, new int[]{ 60, 40, 40, 60 });
        double scale = Math.Pow(2.0, 40);

        using SEALContext context = new SEALContext(parms);
        Console.WriteLine();

        using KeyGenerator keygen = new KeyGenerator(context);
        using PublicKey publicKey = keygen.PublicKey;
        using SecretKey secretKey = keygen.SecretKey;
        using Encryptor encryptor = new Encryptor(context, publicKey);
        using Decryptor decryptor = new Decryptor(context, secretKey);

        using CKKSEncoder encoder = new CKKSEncoder(context);

        using Plaintext xPlain = new Plaintext();
        using Plaintext plainResult = new Plaintext();
        using Ciphertext x1Encrypted = new Ciphertext();

        List<double> result = new List<double>();
        
        Stopwatch stopWatch = new Stopwatch();
        stopWatch.Start();
        
        encoder.Encode(2.718281828459, scale, xPlain);
        encryptor.Encrypt(xPlain, x1Encrypted);
        decryptor.Decrypt(x1Encrypted, plainResult);
        encoder.Decode(plainResult, result);

        stopWatch.Stop();
        
        Console.WriteLine(2.718281828459);
        Console.WriteLine(result[0]);
        Console.WriteLine("Time taken: " + stopWatch.ElapsedMilliseconds.ToString() + "ms");
    }
}

An issue that could arise by timing this way could be noisy data, where the time taken is off by a few milliseconds each run due to external factors. Doing the encryption and decryption many times, and finding the average allows us to get a more accurate measurement of the time taken. To accomplish this, we wrap our encryption/decryption in a for loop that runs 1000 times, and divide the time taken by 1000, to get the average time taken.

In [48]:
class test{
    static void Main(){
        using EncryptionParameters parms = new EncryptionParameters(SchemeType.CKKS);
        ulong polyModulusDegree = 8192;
        parms.PolyModulusDegree = polyModulusDegree;
        parms.CoeffModulus = CoeffModulus.Create(
                        polyModulusDegree, new int[]{ 60, 40, 40, 60 });
        double scale = Math.Pow(2.0, 40);

        using SEALContext context = new SEALContext(parms);
        Console.WriteLine();

        using KeyGenerator keygen = new KeyGenerator(context);
        using PublicKey publicKey = keygen.PublicKey;
        using SecretKey secretKey = keygen.SecretKey;
        using Encryptor encryptor = new Encryptor(context, publicKey);
        using Decryptor decryptor = new Decryptor(context, secretKey);

        using CKKSEncoder encoder = new CKKSEncoder(context);

        using Plaintext xPlain = new Plaintext();
        using Plaintext plainResult = new Plaintext();
        using Ciphertext x1Encrypted = new Ciphertext();

        List<double> result = new List<double>();
        
        Stopwatch stopWatch = new Stopwatch();
        stopWatch.Start();
        
        for (int i = 0; i < 1000; i++){
            encoder.Encode(2.718281828459, scale, xPlain);
            encryptor.Encrypt(xPlain, x1Encrypted);
            decryptor.Decrypt(x1Encrypted, plainResult);
            encoder.Decode(plainResult, result);
        }

        stopWatch.Stop();
        
        Console.WriteLine(2.718281828459);
        Console.WriteLine(result[0]);
        Console.WriteLine("Average Time taken: " + (stopWatch.ElapsedMilliseconds/1000.0).ToString() + "ms");
    }
}

## Working with Xamarin 

To develop on Android, we will be using Xamarin in Visual Studio, which allows us to easily move from our simple C# above to an Android app that accomplishes the same task. To install Xamarin, follow the steps outlined [here](https://docs.microsoft.com/en-us/xamarin/get-started/installation/windows). 

**Note**: If you do not have Visual Studio 2019, you will be instructed to download it.
**Note**: This will *not* work with Linux, so you will have to use either Mac or Windows. I use Windows 10